In [9]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler

In [10]:
spark = SparkSession.builder.appName('project2').getOrCreate()

In [11]:
data = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

In [12]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [13]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [14]:
data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [15]:
assembler = VectorAssembler(inputCols=['Age','Total_Purchase','Num_Sites'],outputCol='features')

In [16]:
output = assembler.transform(data)

In [17]:
final_data= output.select('features','churn')

In [18]:
train_data , test_data = final_data.randomSplit([0.7,0.3])

In [19]:
log_reg_churn = LogisticRegression(labelCol='churn')

In [20]:
fitted_churn_model = log_reg_churn.fit(train_data)

In [21]:
training_sum = fitted_churn_model.summary

In [22]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                655|                655|
|   mean|0.17862595419847327|0.14045801526717558|
| stddev| 0.3833315317577237|0.34772713870208455|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [23]:
pred_and_labels = fitted_churn_model.evaluate(test_data)

In [24]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
| [22.0,11254.38,8.0]|    0|[4.67431528169110...|[0.99075436675997...|       0.0|
|  [25.0,9672.03,8.0]|    0|[4.52248221304405...|[0.98925468757778...|       0.0|
|  [26.0,8939.61,7.0]|    0|[5.67181891723996...|[0.99657020438856...|       0.0|
| [28.0,9090.43,10.0]|    0|[1.94577974388391...|[0.87498573623687...|       0.0|
| [28.0,11128.95,8.0]|    0|[4.2421771044128,...|[0.98582748856591...|       0.0|
| [29.0,10203.18,8.0]|    0|[4.20840334276966...|[0.98534778790586...|       0.0|
| [30.0,7960.64,10.0]|    1|[1.84772335009301...|[0.86385957655648...|       0.0|
|  [30.0,8403.78,7.0]|    0|[5.40284202777752...|[0.99551643001888...|       0.0|
|  [30.0,8874.83,9.0]|    0|[3.00036341175533...|[0.95259054185167...|       0.0|
| [30.0,11575.37

In [25]:
churm_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn')

In [26]:
auc = churm_eval.evaluate(pred_and_labels.predictions)

In [27]:
auc

0.7642938822184105

## Predict on New Data

In [28]:
final_lr_model = log_reg_churn.fit(final_data)

In [29]:
new_customers = spark.read.csv('new_customers.csv', inferSchema=True, header=True)

In [30]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [31]:
test_new_customers = assembler.transform(new_customers)

In [32]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [33]:
final_results = final_lr_model.transform(test_new_customers)

In [34]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [35]:
test_new_customers.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                   6|               6|
|   mean|         null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|                null|            null|
| stddev|         null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|                null|            null|
|    min|Andrew Mccall|              22.0|            100.0|          